In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import json

class Comic:
    def __init__(self, url):
        self.url = url
        self.chapter_links = []
        self.title = ""

    def get_chapter_links(self):
        driver = webdriver.Chrome()  # Make sure you have chromedriver installed and in your PATH
        driver.get(self.url)

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        chapter_links = soup.select('tr.group a')
        chapter_links = [link["href"] for link in chapter_links if "chapter" in link["href"]]

        self.title = soup.find("div", {"class": "justify-between break-words hidden md:block"}).find("h1").text

        driver.quit()

        self.chapter_links = [f"https://comick.app{link}" for link in reversed(chapter_links)]

    def get_image_links(self, url):
        options = webdriver.ChromeOptions()
        options.headless = True
        driver = webdriver.Chrome(options=options)

        driver.get(url)
        html_code = driver.page_source

        soup = BeautifulSoup(html_code, "html.parser")
        div_tags = soup.find("div", {"class": "relative"})

        img_links = []
        if div_tags:
            img_tags = div_tags.find_all("img")
            img_links = [link["src"] for link in img_tags if "comic" in link["src"]]

        driver.quit()

        return img_links

    def save_output_to_json_file(self, filename="test.json"):
        try:
            # Try to load the existing JSON file
            with open(filename, 'r') as json_file:
                data = json.load(json_file)
        except FileNotFoundError:
            # If the file doesn't exist, create a new list
            data = {"page": 1, "results": []}

        new_story = {
            "adult": False,
            "backdrop_path":"meo.comick.pictures/pYMbl.jpg" ,  # Removing "https://"
            "original_language": "en",
            "original_title": self.title,
            "overview":"hi",
            "popularity": 402.295,
            "poster_path": "meo.comick.pictures/pYMbl.jpg",  # Removing "https://"
            "release_date": "",
            "title": self.title,
            "video": True,
            "vote_average": 10,
            "vote_count": 10/10,
            "key": "V-n_w4t9eEU",
            "type": "Trailer",
            "chapters": {"chapters": [self.get_image_links(link) for link in self.chapter_links]}
        }

        # Append the new story to the existing data
        data["results"].append(new_story)

        with open(filename, 'w') as json_file:
            json.dump(data, json_file, indent=2)
        print(f"JSON output saved to '{filename}'")
# Example usage
url = 'https://comick.ink/comic/00-be-the-actor?group=Official'
comic = Comic(url)
comic.get_chapter_links()
comic.save_output_to_json_file()